# Stacks

## 8.1 Implement a Stack with Max API

Design a stach taht includes a max operation, in addition to push and pop. The max method should return the maximum value stored in the stack. Hint: Use additional storage to track the maximum value. 

**Sol:** Suppose we use a single auxiliary variable, M, to record the element that is maximum in the stack. Updating M on pushes is easy: M= max(M,e), where e is the element being pushed. 

However, updating M on opo is very time consuming. If M is the element being popped, we have no way of knowing what the maximum remaining element is, and are forced to consider all the remining elements. 

We can dramatically improve on the time complexity of poping by caching, in essence, trading time for space. Specifically, for each entry in the stack, we cache the maximum stored at or below that entry. Now when we pop, we evict the corresponding cached value. 

In [1]:
import collections

In [2]:
class Stack:
    ElementWithCachedMax = collections.namedtuple('ElementWithCachedMax', 
                                                  ('element', 'max'))
    
    
    def __init__(self) -> None:
        self._element_with_cached_max: List[Stack.ElementWithCachedMax] = []
    
    def empty(self) -> bool:
        return len(self._element_with_cached_max) == 0
    
    def max(self) -> int:
        return self._element_with_cached_max[-1].max
    
    def pop(self) -> int:
        return self._element_with_cached_max.pop().element
    
    def push(self, x: int) -> None:
        self._element_with_cached_max.append(
         self.ElementWithCachedMax(x, x if self.empty() else max(x, self.max())))

Each of the specified methods has time complexity O(1). The additional space complexity is O(n) regardless of the stored keys. 

In [3]:
A = Stack()

In [4]:
A.push(1)

In [5]:
A.push(5)
A.push(0)
A.push(1)
A.push(2)
A.push(2)

In [6]:
A.pop()

2

In [7]:
A.max()

5

In [8]:
A.pop()

2

In [9]:
A.max()

5

In [10]:
def print_stack(A: Stack) -> None:
    while A.empty() is False:
        print(A.pop())

In [11]:
print_stack(A)

1
0
5
1


## 8.2 Evaluate RPN Expressions 

A string is said to be an arithmetical expression in Reverse Polish notation (RPN) if: 

* It is a single digit or a sequence of digits, prefixed with an option -, e.g.m "6", "123", "-42". 
* It is of the form "A, B, o" where A, B are RPN expressions and o is one of "+,-,$\times$, /"

An RPN expression can be evaluated uniquely to an integer, which is determined recursively. The base case corresponds to Rule(1.), which is an integer expressed in base-10 positional system. Rule(2.) on the preceding pagecorresponds to the recursive case, and the RPNs are evaluated in the natural way.

Write a program that takes an arithmetical expression in RPN and returns the number that the expression evaluates to. 

In [46]:
def evaluate(expression: str) -> int:
    intermediate_results: List[int] = []
    delimiter = ','
    operator = {
        '+': lambda y, x: x + y, '-': lambda y, x: x - y,
        '*': lambda y, x: x*y , '/': lambda y, x: x // y
    }
    
    for token in expression.split(delimiter):
        if token in operator:
            intermediate_results.append(operator[token](
                intermediate_results.pop(), intermediate_results.pop()))
        else: # token is a number 
            intermediate_results.append(int(token))
    return intermediate_results[-1]

In [40]:
a = '3,4,+,2,*,1,+'

In [43]:
delimiter = ','
for i in a.split(delimiter):
    print(i)

3
4
+
2
*
1
+


In [47]:
evaluate(a)

15

In [52]:
operator = {
    '+': lambda y, x: x + y, '-': lambda y, x: x - y,
    '*': lambda y, x: x * y, '/': lambda y, x: x /y
}

In [53]:
operator['+'](5,3)

8

In [54]:
operator['/'](5,3)

0.6

Since we perform O(1) computation per character of the string, the time complexity is O(n), where n is the length of the string. 

**Variant:** Solve the same problem for expressions in Polish notation, i.e. when A, B, o is replaced by o, A, B in Rule (2.) on the facing page. 

## 8.3 Is a String Well-formed?

A string over the characters "{,},[,],(,)" is said to be well-formed if the different types of brackets match in the correct order. 

Write a program that tests if a string made up of the characters '(',')','[',']','{','}'is well-formed. 

In [57]:
def is_well_formed(s: str) -> bool:
    left_chars, lookup = [], {'(': ')', '{': '}', '[':']'}
    for c in s:
        if c in lookup:
            left_chars.append(c)
        elif not left_chars or lookup[left_chars.pop()] != c:
            # Unmatched right char or mismatched chars
            return False
    return not left_chars

In [58]:
s = '{[]][[]]}'
is_well_formed(s)

False

In [59]:
s= '({[][]}())'
is_well_formed(s)

True

The time complexity is O(n) since for each character we perform O(1) operations. 

## 8.4 Normalize Pathnames 

A file or directory can be specified via a string called pathname. This string may specify an absolute path, starting from the root, e.g.,/usr/bin/gcc, or a path relative to the current working directory, e.g., scripts/awkscripts.

Write a program which takes a pathname, and returns the shortest equivalent pathname. Assume individual directories and files have names that use only alphanumeric characters. Subdirectory names may be combined using forward slashes (/), the current directory (.) and parent directory (..). 

In [61]:
def shortest_equivalent_path(path: str) -> str:
    if not path:
        raise ValueError('Empty string is not a valid path.')
    
    path_names = [] # Use list as a stack.
    # Special case: starts with '/', which is an aboslute path.
    if path[0] == '/':
        path_names.append('/')
    
    for token in (token for token in path.split('/')
                  if token not in ['.', '']):
        if token == '..':
            if not path_names or path_names == '..':
                path_names.append(token)
            else:
                if path_names[-1] == '/':
                    raise ValueError('Path Error')
                path_names.pop()
        else: # Must be a name
            path_names.append(token)
    
    
    result = '/'.join(path_names)
    return result[result.startswith('//'):] # Avoid starting with '//'

In [63]:
path = 'sc//./../tc/awk/././'
shortest_equivalent_path(path)

'tc/awk'

In [64]:
path = '/usr/bin/../lib/gcc'
shortest_equivalent_path(path)

'/usr/lib/gcc'

In [66]:
path = 'scripts//./../scripts/awkscrits/././'
shortest_equivalent_path(path)

'scripts/awkscrits'

The time complexity is O(n), where n is the length of the pathname. 

## 8.5 Compute Building with a Sunset View 

You are given a series of buildings that have windows facing west. The buildings are in a straight line, and any building which is to the east of a building of equal or greater height cannot view the sunset. 

Design an algorithm that processes buildings in east-to-west order and returns the set of buildings which view the sunset. Each buildings is specified by its height. 

**Sol:** A brute-force approach is to store all buildings in an array. We then do a reverse scan of this array, tracking the running maximum. Any building whose height is less than or equal to the running maximum does not have a sunset view. 

The time and space complexity are both O(n), where n is the number of buildings. 

If a new buidlings is shorter than a building in the current set, then all buildings in the current set which are further to the east cannot be blocked by the new building. This suggests keeping the buildings in a last-in, first-out manner, so that we can terminate earlier. 

Specifically, we use a stack to record buildings that have a view. Each time a building b is processed, if it is taller than the building at the top of the stack, wepop the stack unitl the top of the stack is taller than b--all the buildings thus removed lie to the east of a taller building. 

Althouhg some individual steps may require many pops, each building is pushed and popped at most once. Therefore, the run time to process n buildings is O(n), and the stack always holds precisely the buildings which currently have a view. 

The memory used is O(n), and the bound is tight, even when only one building has a view--consider the input where the west-most building is the tallest, and the remaining n-1 buildings decrease in height from east to west. However, in the best-case, e.g., when buildings appear in increasing height, we use O(1) space. In contrast, the brute-force approach always uses O(n) space. 

In [68]:
def examine_buildings_with_sunset(sequence) -> list:
    BuildingWithHeight = collections.namedtuple('BuildingWithHeight',
                                               ('id', 'height'))
    candidates: List[BuildingWithHeight] = []
    for building_idx, building_height in enumerate(sequence):
        while candidates and building_height >= candidates[-1].height:
            candidates.pop()
        candidates.append(BuildingWithHeight(building_idx, building_height))
    return [c.id for c in reversed(candidates)]

# Queues

## Queues boot camp

In the following program, we implement the basic queue API --enqueue and dequeue--as well as a max-method, which returns the maximum element stored in the queue. The basic idea is n=ti yse cinoisutuib: add a private field that references a library queue object, and forward existing methods (enqueue and dequeue inthis case) to that object. 

In [1]:
class Queue:
    def __init__(self) -> None:
        self._data: Deque[int] = collections.deque()
    
    def enqueue(self, x:int) -> None:
        self._data.append(x)
        
    def dequeue(self) -> int:
        return self._data.popleft()
    
    def max(self) -> int:
        return max(self._data)

The time complexity of enqueue and dequeue are the same as that of the library queue, namely, O(1). The time complexity of finding the maximum is O(n), where n is the number of entries. 

## 8.6 Compute Binary Tree Nodes in Order to Increasing Depth

Binary trees are formally defined in Ch9. In particular, each node in a binary tree has a depth, which is its distance from the root. 

Given a binary tree, return an array consisting of the keys at the same level. Keys should appear in the order of the corresponding nodes' depths, breaking ties from left to right. 

In the following, we use a queue of nodes to store nodes at depth i and a queue of nodes at depth i+1. After all nodes at depth i are processed, we are done with that queue, and can start processing the queue with nodes at depth i+1, putting the depth i+2 nodes in a new queue. 

In [3]:
class BinaryTreeNode:
    def __init__(self, data= None, left= None, right = None):
        self.data = data
        self.left = left
        self.right = right 

In [4]:
def binary_tree_depth_order(tree:BinaryTreeNode) -> list:
    result = []
    if not tree:
        return result # tree is empty 
    
    curr_depth_nodes = [tree]
    while curr_depth_nodes:
        result.append([curr.data for curr in curr_depth_nodes])
        curr_depth_nodes = [
            child for curr in curr_depth_nodes
            for child in (curr.left, curr.right) if child
        ]
        
    return result 

In [5]:
node1 = BinaryTreeNode(314)
node2 = BinaryTreeNode(6)
node3 = BinaryTreeNode(6)
node4 = BinaryTreeNode(271)
node5 = BinaryTreeNode(561)
node6 = BinaryTreeNode(2)
node7 = BinaryTreeNode(271)
node8 = BinaryTreeNode(28)
node9 = BinaryTreeNode(0)
node10 = BinaryTreeNode(3)
node11 = BinaryTreeNode(1)
node12 = BinaryTreeNode(28)
node13 = BinaryTreeNode(17)
node14 = BinaryTreeNode(401)
node15 = BinaryTreeNode(257)
node16 = BinaryTreeNode(641)

In [6]:
node1.left = node2
node1.right = node3
node2.left = node4
node2.right = node5
node3.left = node6
node3.right = node7
node4.left = node8
node4.right = node9
node5.right = node10
node6.right = node11
node7.right = node12
node10.left = node13
node11.left = node14
node11.right = node15
node14.right = node16

In [7]:
binary_tree_depth_order(node1)

[[314], [6, 6], [271, 561, 2, 271], [28, 0, 3, 1, 28], [17, 401, 257], [641]]

Since each node is enqueued and dequeued exactly once, the time complexity is O(n). The space complexity is O(m), where m is the maximum number of nodes at any single depth. 

**Variant:** Write a program which takes as input a binary tree and returns the keys in top down, alternating left-to-right and right-to-left order, starting from left-to-right. 

In [9]:
0^1

1

In [10]:
1^1

0

In [21]:
def binary_tree_depth_order_2(tree:BinaryTreeNode) -> list:
    result = []
    reverse_sign = 0
    if not tree:
        return result # tree is empty 
    
    curr_depth_nodes = [tree]
    while curr_depth_nodes:
        result.append([curr.data for curr in curr_depth_nodes])
        curr_depth_nodes = [
            child for curr in curr_depth_nodes
            for child in (curr.left, curr.right) if child
        ]
        if reverse_sign ==1:
            curr_depth_nodes = reversed(curr_depth_nodes)
        reverse_sign ^= 1
        
    return result 

In [22]:
binary_tree_depth_order_2(node1)

[[314], [6, 6], [271, 2, 561, 271]]

**Variant:** Write a program which takes as input a binary tree with integer keys, and return the average of the keys at each level. 

In [24]:
a = [x**2 for x in range(5)]

In [26]:
sum(a)/len(a)

6.0

In [27]:
def binary_tree_depth_order_avg(tree:BinaryTreeNode) -> list:
    result = []
    if not tree:
        return result # tree is empty 
    
    curr_depth_nodes = [tree]
    while curr_depth_nodes:
        a = [curr.data for curr in curr_depth_nodes]
        result.append(sum(a)/len(a))
        curr_depth_nodes = [
            child for curr in curr_depth_nodes
            for child in (curr.left, curr.right) if child
        ]
        
    return result 

In [28]:
binary_tree_depth_order_avg(node1)

[314.0, 6.0, 276.25, 12.0, 225.0, 641.0]

## 8.7 Implement a Circular Queue

A queue can be implemented using an array and two additional fields, the beginning and the end indices. This structure is sometimes refereed to as a circular queue. 

Implement a queue API using an array for storing elements. Your API should include a constructor function, which takes as argument the initial capacity of the queue, enqueue and dequeue functions and a function which returns the number of elements stored. Implement dynamic resizing to support storing an arbitrarily large number of elements. 

In [42]:
S = '13 DUP POP'
stack = []
for s in S.split():
    if s.isdigit():
        print(int(s))
        stack.append(int(s))


13


In [44]:
class Queue:
    SCALE_FACTOR = 2
    
    def __init__(self, capacity: int) -> None:
        self._entires = [0]*capacity 
        self._head = self._tail = self._num_queue_elements = 0
        
    def enqueue(self, x: int) -> None:
        if self._num_queue_elements == len(self.__entries): # Needs to resize
            # Make the queue elements appear consecutively.
            self._entries = (
                self._entries[self._head:] + self._entries[:self._head])
            # Resets head and tail.
            self._head, self._tail = 0, self._num_queue_elements
            self._entries += [0]*(
                len(self._entries) * Queue.SCALE_FACTOR - len(self._entries))
            
            self._entries[self._tail] = x
            self._tail = (self._tail +1) % len(self._entries)
            self._num_queue_elements += 1
            
    def dequeue(self) -> int:
        self._num_queue_elements -= 1
        result = self._entries[self._head]
        self._head = (self._head + 1) % len(self._entries)
        return result
    
    def size(self) -> int:
        return self._num_queue_elements
        

The time complexity of dequeue is O(1), and the amortized time complexity of enqueue is O(1). 

## 8.8 Implement a Queue Using Stacks 

Queue insertion and deletion follows first-in, first-out semantics; stack insertion and deletion is last-in, first-out. 

How would you implement a queue given a library implementing stacks?

**Sol:** It is impossible to solve this problem with a single stack. In essence, we are using the first stack for enqueue and the second for dequeue. 

In [45]:
class Queue:
    def __init__(self) -> None:
        self._enq: list = []
        self._deq: list = []
    
    def enqueue(self, x:int) -> None:
        self._enq.append(x)
        
    def dequeue(self) -> int:
        if not self._deq:
            # Transfers the elements in _enq to _deq 
            while self._enq:
                self._deq.append(self._enq.pop())
        return self._deq.pop()

In [67]:
queue1 = Queue()

In [68]:
queue1.enqueue(1)

In [69]:
queue1.enqueue(3)

In [70]:
queue1.enqueue(5)

In [71]:
queue1.enqueue(7)

In [72]:
queue1.enqueue(9)

In [75]:
def print_queue(q: Queue):
    while len(q._deq)+len(q._enq) >=1:
        print(q.dequeue())

In [76]:
print_queue(queue1)

1
3
5
7
9


This approach takes O(m) time for m operations, which can be seen from the fact that each element is pushed no more than twice and poped no more than twice. 

## 8.9 Implement a Queue with MAX API

Implement a queue with enqueue, dequeue, and max operations. The max operation returns the maximum element currently stored in the queue.

**Sol:** A brute-force solution is to track the current maximum. The current maximum has to be updated on both enqueue and dequeue. Updating the current maximum on enqueue is trivial and fast-just compare the enqueued value with the current maximum. However, updating the current maximum on dequeue is slow--we must examine every single remaining element, which takes O(n) time, where n is the size of the queue.

Consider an element s in the queue that has the property that it entered the queue before a later element, b, which is greater than s. Since s will be dequeued before b, s can never in the future become the maximum element stored in the queue, regardless of the subsequent enqueues and dequeues. 

The key to a faster implementation of a queue-with-max is to eliminate elements like s from consideration. We do this by maintaining the set of entries in the queue taht have no later entry in the queue greater than them in a separate deque. Elements in the deque will be ordered by their position in the queue, with the candidate closest to the head of the queue appearing first. Since each entry in the deque is greater than or equal to its successors, the largest element in the queue is at the head of the deque. 

We now briefly describe how to update the deque on queue updates. If the queue is dequeued, and if the element just dequeued is at the deque's head, we pop the deque from its head; otherwise the deque remains unchanged. When we add an entry to the queue, we iteratively evict from the deque's tail unitl the element at the tail is greater than or equal to the entry being enqueued, and then add the new entry to the deque's tail. 

In [79]:
import collections

In [80]:
class QueueWithMax:
    def __init__(self):
        self._entries: Deque[Any] = collections.deque()
        self._candidates_for_max: Deque[Any] = collections.deque()
            
    def enqueue(self, x):
        self._entries.append(x)
        # Eliminate dominated elements in _candidates_for_max.
        while self._candidates_for_max and self._candidates_for_max[-1] <x:
            self._candidates_for_max.pop()
        self._candidates_for_max.append(x)
        
    def dequeue(self):
        result = self._entries.popleft()
        if result == self._candidates_for_max[0]:
            self._candidates_for_max.popleft()
        return result 
    
    def max(self):
        return self._candidates_for_max[0]

In [81]:
A = QueueWithMax()

In [82]:
A.enqueue(3)

In [83]:
A.enqueue(1)
A.enqueue(3)
A.enqueue(2)
A.enqueue(0)

In [84]:
print(A._entries)

deque([3, 1, 3, 2, 0])


In [85]:
print(A._candidates_for_max)

deque([3, 3, 2, 0])


In [87]:
A.dequeue()
print(A._candidates_for_max)

deque([3, 2, 0])


In [88]:
A.dequeue()
print(A._entries)
print(A._candidates_for_max)

deque([3, 2, 0])
deque([3, 2, 0])


In [89]:
A.enqueue(1)
print(A._entries)
print(A._candidates_for_max)

deque([3, 2, 0, 1])
deque([3, 2, 1])


In [90]:
A.enqueue(2)
print(A._entries)
print(A._candidates_for_max)

deque([3, 2, 0, 1, 2])
deque([3, 2, 2])


In [91]:
A.enqueue(4)
print(A._entries)
print(A._candidates_for_max)

deque([3, 2, 0, 1, 2, 4])
deque([4])


In [92]:
A.dequeue()
print(A._entries)
print(A._candidates_for_max)

deque([2, 0, 1, 2, 4])
deque([4])


In [93]:
A.enqueue(4)
print(A._entries)
print(A._candidates_for_max)

deque([2, 0, 1, 2, 4, 4])
deque([4, 4])


Each dequeue operation has O(1) time complexity. A single enqueue operation may entail many ejections from the deque. However, the amortized time complexity of n enqueues and dequeues is O(n), since an element can be added and removed from the deque no more than once. The max opreration is O(1) since it consists of returning the lement at the head of the deque. 

We can solve the queue-with-max design by modeling a queue with two stacks-with-max. This approach feels unnatural compared to the one presented above. 

In [100]:
class QueueWithMax2:
    def __init__(self) -> None:
        self._enqueue, self._dequeue = Stack(), Stack()
        
    def enqueue(self, x:int) -> None:
        self._enqueue.push(x)
    
    def dequeue(self) -> int:
        if self._dequeue.empty():
            while not self._enqueue.empty():
                self._dequeue.push(self._enqueue.pop())
        return self._dequeue.pop()
    
    def max(self) -> int:
        if not self._enqueue.empty():
            return self._enqueue.max() if self._dequeue.empty() else max(
                self._enqueue.max(), self._dequeue.max())
        return self._dequeue.max()

In [102]:
A = QueueWithMax2()
A.enqueue(3)
A.enqueue(1)
A.enqueue(3)
A.enqueue(2)
A.enqueue(0)

In [103]:
A.max()

3

In [104]:
A.dequeue()
A.max()

3

In [105]:
A.dequeue()
A.max()

3

In [106]:
A.enqueue(2)
A.max()

3

In [107]:
A.enqueue(4)
A.max()

4

In [108]:
A.enqueue(4)
A.max()

4

In [109]:
A.enqueue(5)
A.max()

5

Since the stack-with-max has O(1) amortized time complexity for push, pop, and max, and the queue from two stacks has O(1) amortized time complexity for enqueue and dequeue, this approach has O(1) amortized time complexity for enqueue, dequeue, and max. 